# Step 1 : Download YouTube Video's Audio

In [ ]:
! pip install youtube_dl
#Installing youtube_dl for downloading the audio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 8.4 MB/s 


In [ ]:
# VIDEO_URL = "https://www.youtube.com/watch?v=hWLf6JFbZoo" #obama

In [ ]:
import youtube_dl
ydl_opts = {'outtmpl': '/content/ytaudio.webm'}
with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    ydl.download(['https://www.youtube.com/watch?v=hWLf6JFbZoo'])
#downloading in webm format, it can take time based on internet speed and length of the video.
# since it's a mini project, keep the video length small, try using the same video.

[youtube] hWLf6JFbZoo: Downloading webpage
[download] Destination: /content/ytaudio.f248.webm
[download] 100% of 43.33MiB in 09:12
[download] Destination: /content/ytaudio.f251.webm
[download] 100% of 3.25MiB in 00:41
[ffmpeg] Merging formats into "/content/ytaudio.webm"
Deleting original file /content/ytaudio.f248.webm (pass -k to keep)
Deleting original file /content/ytaudio.f251.webm (pass -k to keep)


In [ ]:
! ffmpeg -i ytaudio.webm -acodec pcm_s16le -ar 16000 ytaudio.wav
#converting webm to wav

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

# English ASR with HuggingSound

In [ ]:
!pip install huggingsound -q 

     |████████████████████████████████| 25.3 MB 1.3 MB/s 
     |████████████████████████████████| 3.4 MB 70.6 MB/s 
     |████████████████████████████████| 4.2 MB 74.3 MB/s 
     |████████████████████████████████| 312 kB 62.7 MB/s 
     |████████████████████████████████| 140 kB 79.2 MB/s 
     |████████████████████████████████| 86 kB 7.1 MB/s 
     |████████████████████████████████| 1.1 MB 55.7 MB/s 
     |████████████████████████████████| 212 kB 67.5 MB/s 
     |████████████████████████████████| 596 kB 71.8 MB/s 
     |████████████████████████████████| 50 kB 8.4 MB/s 
     |████████████████████████████████| 127 kB 72.9 MB/s 
     |████████████████████████████████| 6.6 MB 54.1 MB/s 
     |████████████████████████████████| 271 kB 64.2 MB/s 
     |████████████████████████████████| 94 kB 4.1 MB/s 
     |████████████████████████████████| 144 kB 80.9 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source 

In [ ]:
from huggingsound import SpeechRecognitionModel

In [ ]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
#Enable GPU => Runtime > Change Runtime type > Hardware Accelerator > GPU > Save.

In [ ]:
device

'cuda'

In [ ]:
model = SpeechRecognitionModel("jonatasgrosman/wav2vec2-large-xlsr-53-english", device = device)
#This can take a bit of a time.

NameError: ignored

# Audio Chunking

If you straightaway start the transcripting, you can encounter the  OOM Error by the SpeechRecognitionModel, i.e. Out Of Memory error if the video size > 1 min. As there is limited RAM here in Google Colab, the whole file at once can't be processed, to do so you'll have to but Google Colab Pro or buy GPU.
So we'll be performing audio chunking to avoid OOM, which is nothing but splitting the audio file into multiple smaller chunks and writing them into individual files. 

In [ ]:
import librosa
#We'll be using a library named librosa for chunking process, it'll read the file and cut the stream to 30 secs

In [ ]:
input_file = '/content/ytaudio.wav'

In [ ]:
print(librosa.get_samplerate(input_file))

# Stream over 30 seconds chunks rather than load the full file
stream = librosa.stream(
    input_file,
    block_length=30,
    frame_length=16000,
    hop_length=16000
)

16000


In [ ]:
import soundfile as sf


In [ ]:
for i,speech in enumerate(stream):
  sf.write(f'{i}.wav', speech, 16000)

#making i number of individual files of 30 secs


In [ ]:
i

6

# Audio Transcription / ASR / Speech to Text

In [ ]:
audio_path =[]
for a in range(i+1):
  audio_path.append(f'/content/{a}.wav') #nameing the newly written files as 0,1,2,3,4... and assigning them to the empty list audio_path

In [ ]:
audio_path

['/content/0.wav',
 '/content/1.wav',
 '/content/2.wav',
 '/content/3.wav',
 '/content/4.wav',
 '/content/5.wav',
 '/content/6.wav']

In [ ]:
transcriptions = model.transcribe(audio_path) #transcription from the speechrecognitionmodel passig the list

100%|██████████| 7/7 [00:07<00:00,  1.14s/it]


In [ ]:
full_transcript = ' ' #empty string to merge all the transcription from the audio chunks

In [ ]:
for item in transcriptions:
  full_transcript += ''.join(item['transcription'])

In [ ]:
len(full_transcript) #size of the text retrieved from the transcription

2208

In [ ]:
full_transcript

" for even as we celebrate tonight we know the challenges the tomorrow will bring are the greatest timorlitai stand here knowing that my story is part of the larger american story tti owe a debt to all of those who came before me and that in no other country on earth is my story even possiblei am te sonblackman from kenya and white woman from kansas i am married to a black american who carries within her the blood of slaves and slaveownersi have brothers sisters nieces nephews uncles and cousin of every race and every hugh scattered across three continents these people are part of meand they are part of america this country that i lovei've come here to cairoo seek a new beginning between the united states and muslims around the worldone based on mutual interest and mutual respect just as muslims do not fit a cruidstereotype america is not the cruid stereotpe of a self-interested empireto say that force may sometimes be necessary is not a call to synicism it is a recognition of historym

# Text Summarization 

In [ ]:
from transformers import pipeline #using NLP transformers pipeline

In [ ]:
summarization = pipeline('summarization')

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)


Downloading:   0%|          | 0.00/1.76k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

In [ ]:
summarized_text = summarization(full_transcript)

In [ ]:
summarized_text[0]['summary_text']

' President Obama spoke at a memorial service in honor of the courage of ordinary americans willing to endure billy clotesnchasening rodteer gas and trampling hoof men and women who despite the gush of blood and splintard bone would stay true to their norse star .'